In [2]:
import sys
sys.path.append('..//')
from ioMicro import *

In [1]:
genes = ['NeuroD1', 'FoxJ1', 'Gfap', 'GAD1', 'NRGN', 'AQP4', 'PTBP1', 'PTBP2', 'REST']

In [3]:
save_fld_mos = r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023\Mosaic_Analysis'

In [51]:
def load_imsm(gene,ifov,save_fld_mos = r'\\192.168.0.127\Raw_data\Roy\HumanSVZMER_6_26_2023\Mosaic_Analysis'):

    mosaic_fl = save_fld_mos+os.sep+str(ifov).zfill(4)+'__'+gene+'.npz'
    if os.path.exists(mosaic_fl):
        im_nc = np.load(mosaic_fl)['im']
        Xh = np.load(mosaic_fl)['Xh']
        im_dapi = np.load(mosaic_fl)['im_dapi_']
        x,y,icol = np.load(mosaic_fl)['xycol']
        return im_nc,Xh,im_dapi,icol,x,y
    return [None]*6
def get_mosaic(gene = 'Ifit1',tag='None',ifovs = [1277, 1278, 1273, 1276, 1252, 1251, 1274, 1275, 1253, 1304, 1305, 1306]):
    ims,xy,txy = [],[],[]
    #fovs = [999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1098, 1099, 1100, 1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110, 1111, 1112, 1113, 1114, 1115, 1116, 1117, 1118, 1119, 1120, 1121, 1122, 1123, 1124, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142, 1143, 1144, 1145, 1146, 1147, 1148, 1149, 1150, 1151, 1152, 1153, 1154, 1155, 1156, 1157, 1158, 1159, 1160, 1161, 1162, 1163, 1164, 1165, 1166, 1167, 1168, 1169, 1170, 1171, 1172, 1173, 1174, 1175, 1176, 1177, 1178, 1179, 1180, 1181, 1182, 1183, 1184, 1185, 1186, 1187, 1188, 1189, 1190, 1191, 1192, 1193, 1194, 1195, 1196, 1197, 1198, 1199, 1200, 1201, 1202, 1203, 1204, 1205, 1206, 1207, 1208, 1209, 1210, 1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1314, 1315, 1316, 1317, 1401, 1402, 1403, 1404]#np.unique([int(os.path.basename(fl).split('_')[0]) for fl in flsanalz])
    for ifov in tqdm(ifovs):
        im_nc,Xh,im_dapi,icol,x,y = load_imsm(gene,ifov)
        if im_dapi is not None:
            if tag=='dapi':
                im = im_dapi.T[::-1,::1]
            else:
                im = im_nc.T[::-1,::1]
            
            ims.append(im)
            xy.append([x,y])
            txy.append([x,y])
        #txy.append([tx_,ty_])
    xy = np.array(xy)
    #ims = np.array(ims)
    txy = np.array(txy)


    xy_ = ((xy-np.min(xy,axis=0))/0.10833).astype(int)
    sx,sy = ims[0].shape
    sf = np.max(xy_,axis=0)+[3*sx+1,3*sy+1]
    startx,starty = sx,sy
    imf = np.zeros(sf,dtype=np.float32)
    ifov_coords = []
    for im,(x,y),(txT,tyT)in zip(ims,xy_,txy):
        imT = imf[(startx+x):(startx+x+sx-2),(starty+y):(starty+y+sy-2)]
        im_to_add = im.T[::-1,::1][1:-1,1:-1]
        sx,sy = im_to_add.shape
        ty = 0
        tx= 0
        imf[(startx+tx+x):(startx+tx+x+sx),(starty+ty+y):(starty+ty+y+sy)] = im_to_add
        ifov_coords.append([((startx+tx+x)+(startx+tx+x+sx))/2,((starty+ty+y)+(starty+ty+y+sy))/2])
    return imf,ifov_coords


ifovs = [264,265,266,275,276,277,279,280,281,
 263,268,267,274,273,278,283,282,311,
 257,269,270,271,272,285,284,309,310,
 256,251,250,249,287,286,307,308,220]
gene = 'AQP4'
dic_im = {}
for gene in list(genes):
    imf,ifov_coords = get_mosaic(gene = gene,ifovs = ifovs)
    dic_im[gene] = imf
imf,ifov_coords = get_mosaic(gene = gene,tag='dapi',ifovs = ifovs)
dic_im['dapi'] = imf

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:11<00:00,  3.08it/s]


In [52]:
imf,ifov_coords = get_mosaic(gene = gene,tag='dapi',ifovs = ifovs)
dic_im['dapi'] = imf

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:10<00:00,  3.55it/s]


In [54]:
import napari
v = napari.Viewer()
for name in dic_im:
    v.add_image(dic_im[name].T,contrast_limits=[0,5000],name=name,visible=False,blending='additive')

'dapi'

In [47]:
get_mosaic(gene = gene,ifovs = ifovs)

220

In [ ]:
imf,ifov_coords = get_mosaic(gene = gene,ifovs = ifovs)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 8244.5, 14994.0), zoom=0.025681343492076996, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 16490.0, 1.0), (0.0, 29989.0, 1.0)), current_step=(8245, 14994), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'Image' at 0x1c186489ee0>], scale_bar=ScaleBar(visible=False, colored=False, color=array([1., 0., 1., 1.], dtype=float32), ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, box=False, box_color=array([0. , 0. , 0. , 0.6], dtype=float32), unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ], dtype=float32), font_size=10.0, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=

In [18]:
ifov =264


In [19]:

for ifov in tqdm(ifovs):
    im_nc,Xh,im_dapi,icol,x,y = load_imsm(gene,ifovs)
    

100%|████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 2886.15it/s]


In [ ]:
def get_mosaic(gene = 'Ifit1',fovs = [1277, 1278, 1273, 1276, 1252, 1251, 1274, 1275, 1253, 1304, 1305, 1306],subtract_bk=True):
    ims,xy,txy = [],[],[]
    #fovs = [999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1098, 1099, 1100, 1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110, 1111, 1112, 1113, 1114, 1115, 1116, 1117, 1118, 1119, 1120, 1121, 1122, 1123, 1124, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142, 1143, 1144, 1145, 1146, 1147, 1148, 1149, 1150, 1151, 1152, 1153, 1154, 1155, 1156, 1157, 1158, 1159, 1160, 1161, 1162, 1163, 1164, 1165, 1166, 1167, 1168, 1169, 1170, 1171, 1172, 1173, 1174, 1175, 1176, 1177, 1178, 1179, 1180, 1181, 1182, 1183, 1184, 1185, 1186, 1187, 1188, 1189, 1190, 1191, 1192, 1193, 1194, 1195, 1196, 1197, 1198, 1199, 1200, 1201, 1202, 1203, 1204, 1205, 1206, 1207, 1208, 1209, 1210, 1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1314, 1315, 1316, 1317, 1401, 1402, 1403, 1404]#np.unique([int(os.path.basename(fl).split('_')[0]) for fl in flsanalz])
    for ifov in tqdm(fovs):
        im,x,y,icol,tx_,ty_ = load_subtract_bk(gene,ifov,subtract_bk=subtract_bk)
        ims.append(im)
        xy.append([x,y])
        txy.append([tx_,ty_])
    xy = np.array(xy)
    #ims = np.array(ims)
    txy = np.array(txy)


    xy_ = ((xy-np.min(xy,axis=0))/0.10833).astype(int)
    sx,sy = ims[0].shape
    sf = np.max(xy_,axis=0)+[3*sx+100,3*sy+100]
    startx,starty = sx,sy
    imf = np.zeros(sf,dtype=np.uint8)
    ifov_coords = []
    for im,(x,y),(txT,tyT)in zip(ims,xy_,txy):
        imT = imf[(startx+x):(startx+x+sx-2),(starty+y):(starty+y+sy-2)]
        im_to_add = im.T[::-1,::1][1:-1,1:-1]
        sx,sy = im_to_add.shape

        #tx_,tx__,ty_,ty__ = get_txy(-tx,-ty)
        #tx = ty_
        #ty = tx_
        
        tx_,tx__,ty_,ty__ = get_txy(-txT,-tyT)
        #tx = -100
        ty = tx_
        tx= ty_
        imf[(startx+tx+x):(startx+tx+x+sx),(starty+ty+y):(starty+ty+y+sy)] = im_to_add
        ifov_coords.append([((startx+tx+x)+(startx+tx+x+sx))/2,((starty+ty+y)+(starty+ty+y+sy))/2])
    return imf,ifov_coords

fovs = [1277, 1278, 1273, 1276, 1252, 1251, 1274, 1275, 1253, 1304, 1305, 1306]
fovs = [999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1098, 1099, 1100, 1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110, 1111, 1112, 1113, 1114, 1115, 1116, 1117, 1118, 1119, 1120, 1121, 1122, 1123, 1124, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142, 1143, 1144, 1145, 1146, 1147, 1148, 1149, 1150, 1151, 1152, 1153, 1154, 1155, 1156, 1157, 1158, 1159, 1160, 1161, 1162, 1163, 1164, 1165, 1166, 1167, 1168, 1169, 1170, 1171, 1172, 1173, 1174, 1175, 1176, 1177, 1178, 1179, 1180, 1181, 1182, 1183, 1184, 1185, 1186, 1187, 1188, 1189, 1190, 1191, 1192, 1193, 1194, 1195, 1196, 1197, 1198, 1199, 1200, 1201, 1202, 1203, 1204, 1205, 1206, 1207, 1208, 1209, 1210, 1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1314, 1315, 1316, 1317, 1401, 1402, 1403, 1404]#np.unique([int(os.path.basename(fl).split('_')[0]) for fl in flsanalz])

